In [1]:
from ftl_app.ftl_core.ftl_model import run_ftl_simulation
import json

accuracy, loss, leakage, latency, epsilon = run_ftl_simulation(num_rounds=10, num_devices=10, noise_multiplier=1.1)
with open('ftl_app/precomputed/metrics.json', 'w') as f:
    json.dump({
        'accuracy': accuracy, 'loss': loss, 'leakage': leakage, 
        'latency': latency, 'epsilon': epsilon
    }, f)
print("Precomputation completed successfully!")

2025-04-04 23:24:20.173914: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 23:24:20.397362: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-04 23:24:20.397466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-04 23:24:20.397532: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 23:24:20.483498: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 23:24:20.491565: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Precomputation completed successfully!
